# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from scipy.stats import loguniform
from sklearn.svm import SVR
import pandas as pd
import timeit
import time
from joblib import dump, load
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt, random

# Data

In [4]:
data_type = "1"

In [3]:
df = pd.read_csv("train/{}.csv".format(data_type))
df.drop(["c/b", "F"], axis=1, inplace=True)
d = df.to_numpy()

d_le = np.load("train/{}_le.npy".format(data_type))
d_g = np.load("train/{}_g.npy".format(data_type))

d.shape, d_le.shape, d_g.shape

((408, 4), (360, 4), (48, 4))

# Hyperparameter Optimisation

In [3]:
# defining parameter range
param_grid = {'C': list(10. ** np.arange(-3, 8)),
              'gamma': list(10. ** np.arange(-5, 4)),
              'kernel': ['rbf']}
 
# define search
grid = RandomizedSearchCV(SVR(), param_grid, n_iter=50, scoring='neg_mean_squared_error', n_jobs=-1, verbose=2)
# fitting the model for grid search
grid.fit(d[:,:3], d[:,3])

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END .....................C=0.001, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END .................C=10000.0, gamma=1e-05, kernel=rbf; total time=   0.0s
[CV] END ...............C=10000000.0, gamma=10.0, kernel=rbf; total time=   0.0s
[CV] END ...................C=100.0, gamma=1e-05, kernel=rbf; total time=   0.0s
[CV] END .....................C=1.0, gamma=0.001, kernel=rbf; total time=   0.0s
[CV] END ......................C=1.0, gamma=10.0, kernel=rbf; total time=   0.0s
[CV] END ......................C=1.0, gamma=10.0, kernel=rbf; total time=   0.0s
[CV] END ...................C=10.0, gamma=0.0001, kernel=rbf; total time=   0.0s
[CV] END ...................C=10.0, gamma=0.0001, kernel=rbf; total time=   0.0s
[CV] END ...............C=1000000.0, gamma=1e-05, kernel=rbf; total time=   0.1s
[CV] END ...............C=1000000.0, gamma=1e-05, kernel=rbf; total time=   0.1s
[CV] END ...............C=1000000.0, gamma=0.00

[CV] END ...................C=10.0, gamma=1000.0, kernel=rbf; total time=   0.0s
[CV] END ...................C=10.0, gamma=1000.0, kernel=rbf; total time=   0.0s
[CV] END ...................C=10.0, gamma=1000.0, kernel=rbf; total time=   0.0s
[CV] END ...................C=10.0, gamma=1000.0, kernel=rbf; total time=   0.0s
[CV] END .......................C=1.0, gamma=1.0, kernel=rbf; total time=   0.0s
[CV] END .......................C=1.0, gamma=1.0, kernel=rbf; total time=   0.0s
[CV] END .......................C=1.0, gamma=1.0, kernel=rbf; total time=   0.0s
[CV] END .......................C=1.0, gamma=1.0, kernel=rbf; total time=   0.0s
[CV] END .......................C=1.0, gamma=1.0, kernel=rbf; total time=   0.0s
[CV] END .....................C=10.0, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .....................C=10.0, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END .....................C=10.0, gamma=0.01, kernel=rbf; total time=   0.0s
[CV] END ...................

RandomizedSearchCV(estimator=SVR(), n_iter=50, n_jobs=-1,
                   param_distributions={'C': [0.001, 0.01, 0.1, 1.0, 10.0,
                                              100.0, 1000.0, 10000.0, 100000.0,
                                              1000000.0, 10000000.0],
                                        'gamma': [1e-05, 0.0001, 0.001, 0.01,
                                                  0.1, 1.0, 10.0, 100.0,
                                                  1000.0],
                                        'kernel': ['rbf']},
                   scoring='neg_mean_squared_error', verbose=2)

In [4]:
# print best parameter after tuning
print(grid.best_params_)
 
# print how our model looks after hyper-parameter tuning
print(grid.best_estimator_)

{'kernel': 'rbf', 'gamma': 0.1, 'C': 100000.0}
SVR(C=100000.0, gamma=0.1)
[CV] END .................C=100000.0, gamma=10.0, kernel=rbf; total time=   0.0s
[CV] END ................C=10000000.0, gamma=0.1, kernel=rbf; total time= 6.9min
[CV] END .................C=100000.0, gamma=10.0, kernel=rbf; total time=   0.0s
[CV] END ................C=10000000.0, gamma=0.1, kernel=rbf; total time= 7.8min
[CV] END .....................C=0.001, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ................C=10000000.0, gamma=0.1, kernel=rbf; total time= 8.8min
[CV] END .................C=100000.0, gamma=10.0, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=1e-05, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=1e-05, kernel=rbf; total time=   0.0s
[CV] END .....................C=0.1, gamma=1e-05, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.01, gamma=0.1, kernel=rbf; total time=   0.0s
[CV] END ......................C=0.

# RBF Fitting

In [4]:
# All Dataset
t0 = time.time()
reg_all = SVR(kernel="rbf", gamma=0.1, C=100000)
reg_all.fit(d[:,:-1], d[:,-1])
t1 = time.time()
print("RBF train time: {} secs".format(t1-t0))

RBF train time: 0.9922046661376953 secs


In [5]:
# a/c > 1
t0 = time.time()
reg_g = SVR(kernel="rbf", gamma=0.1, C=100000)
reg_g.fit(d_g[:,:-1], d_g[:,-1])
t1 = time.time()
print("RBF train time: {} secs".format(t1-t0))

RBF train time: 0.0009007453918457031 secs


In [6]:
# a/c <= 1
t0 = time.time()
reg_le = SVR(kernel="rbf", gamma=0.1, C=100000)
reg_le.fit(d_le[:,:-1], d_le[:,-1])
t1 = time.time()
print("RBF train time: {} secs".format(t1-t0))

RBF train time: 0.9146866798400879 secs


# Results

In [2]:
reg_all = load("models/rbf_all.joblib")
reg_le = load("models/rbf_le.joblib")
reg_g = load("models/rbf_g.joblib")

In [5]:
df = pd.read_csv("test/{}.csv".format(data_type))
df.drop(["c/b", "F"], axis=1, inplace=True)
d = df.to_numpy()

d_le = np.load("test/{}_le.npy".format(data_type))
d_g = np.load("test/{}_g.npy".format(data_type))

d.shape, d_le.shape, d_g.shape

((136, 4), (116, 4), (20, 4))

In [7]:
def error(reg, data):
    t0 = time.time()
    result = reg.predict(data[:,:-1])
    t1 = time.time()
    MAE = np.mean(np.power(result-data[:,-1],2))
    return MAE, t1-t0

mse, t = error(reg_all, d)
print("MAE all: ", mse)
print("Prediction time: ", t)

mse_le, t = error(reg_le, d)
print("MAE a/c<=1: ", mse_le)
print("Prediction time: ", t)

mse_g, t = error(reg_g, d)
print("MAE a/c>1: ", mse_g)
print("Prediction time: ", t)

MAE all:  0.0048922922853430395
Prediction time:  0.0007302761077880859
MAE a/c<=1:  0.284068883855525
Prediction time:  0.00042748451232910156
MAE a/c>1:  0.47064689153970596
Prediction time:  0.00037026405334472656


In [13]:
dump(reg_all, 'models/rbf_all.joblib')
dump(reg_le, 'models/rbf_le.joblib')
dump(reg_g, 'models/rbf_g.joblib')

['models/rbf_g.joblib']

## Fitting Equations

In [10]:
def sort(data):
    models = []
    model = np.unique(data[:,[0,1,2]], axis=0)
    
    for i in model:
        models.append(data[np.where((data[:,[0,1,2]] == i).all(axis=1))])

    return models 

def compare(model):
    results_all = []
    results_sep = []
    
    result = np.zeros((len(model),1))
    model = np.delete(model, [2,4], 1)

    result[:,0] = reg_all.predict(model[:,:-1])
    results_all.append(result)

    # a/c separated
    result = np.zeros((len(model),1))
    if model[0,0] <= 1:
        result[:,0] = reg_le.predict(model[:,:-1])

    elif model[0,0] > 1:
        result[:,0] = reg_g.predict(model[:,:-1])

    else:
        print("Something wrong!")

    results_sep.append(result)
        
    return results_all, results_sep



def plot(model_num):
    results_all, results_sep = compare(models[model_num])
    plt.scatter(models[model_num][:,-3], models[model_num][:,-1], label="Ground Truth")
    plt.plot(models[model_num][:,-3], results_all[0][:,0], label="NN all", color='red')
    plt.plot(models[model_num][:,-3], results_sep[0][:,0], label="NN separated", color='green')
    plt.title("a/c={}; a/t={}; c/b={}".format(models[model_num][0,0],
                                             models[model_num][0,1],
                                             models[model_num][0,2]))
    plt.xlabel("phi")
    plt.ylabel("Mg")
    plt.legend()
    plt.show()
    return

In [11]:
# Loading test dataset
df = pd.read_csv("1_RN_data.csv")
d = df.to_numpy()

models = sort(d)
len(models)

32

In [14]:
interact(plot, model_num=(1,len(models)-1,1));

interactive(children=(IntSlider(value=16, description='model_num', max=31, min=1), Output()), _dom_classes=('w…